In [333]:
import numpy as np
import random
from copy import deepcopy

In [334]:
class SnakeNode():
    def __init__(self, parent, x, y):
        self.parent = parent
        self.x = x
        self.y = y
    def SetX(self, newx):
        self.x = newx
    def SetY(self, newy):
        self.y = newy
    def SetParent(self, newparent):
        self.parent = newparent
    def Pos(self):
        return (self.x, self.y)
    def Index(self):
        return (self.y, self.x)

In [335]:
class Snake():

    def __init__(self, HeadX, HeadY):
        self.head = SnakeNode(None, HeadX, HeadY)
        self.tail = self.head

    def MoveBodyForward(self):
        currentnode = self.tail
        while (currentnode.parent != None):
            parentpos = currentnode.parent.Pos()
            currentnode.SetX(parentpos[0])
            currentnode.SetY(parentpos[1])
            currentnode = currentnode.parent

    def Move(self, direction):
        (oldtailx, oldtaily) = self.tail.Pos()
        self.MoveBodyForward()
        headpos = self.head.Pos()
        if direction == 0:
            self.head.SetY(headpos[1] - 1)
        elif direction == 1:
            self.head.SetX(headpos[0] + 1)
        elif direction == 2:
            self.head.SetY(headpos[1] + 1)
        elif direction == 3:
            self.head.SetX(headpos[0] - 1)
        
        return (oldtailx, oldtaily, *self.head.Pos())
    
    def NewHead(self, headx, heady):
        newhead = SnakeNode(None, headx, heady)
        self.head.SetParent(newhead)
        self.head = newhead

    def Head(self):
        return self.head
    def Tail(self):
        return self.tail

In [336]:
class Game():
    x = 1
    def __init__(self, weights, snakecount):
        self.foodval = 5
        self.headval = 10
        self.bodyval = 2
        self.snakes = []
        self.features = np.zeros(5, dtype= int)
        #self.theta = np.ones(np.shape(self.features), dtype= int)
        self.theta = weights
        self.statevalue = 0
        self.obstacleval = 1
        self.length = 1
        self.board = np.zeros((20,20), dtype=int)
        self.SpawnObstacle()
        
        self.SpawnSnake(snakecount)
        #snake2 = self.SpawnSnake2()
        #self.snakes.append(snake1)
        #self.snakes.append(snake2)
        self.SpawnFood()

    def SpawnSnake(self, snakecount):
        for i in range(snakecount):
            snakespawns = []
            for index, cell in np.ndenumerate(self.board):
                if cell == 0:
                    snakespawns.append(index)
            self.snakespawn = random.choice(snakespawns)
            startx = self.snakespawn[0]
            starty = self.snakespawn[1]
            #self.board[startx, starty] = self.headval
            self.tempsnake = Snake(startx, starty)
            self.snakes.append(deepcopy(self.tempsnake))
        #return self.snakes


    def SpawnObstacle(self):
        obstaclespawns = []
        obstaclespawns.append(np.random.randint(1, 19, size=(2)))
        obstaclespawns.append(np.random.randint(1, 19, size=(2)))
        obstaclespawns.append(np.random.randint(1, 19, size=(2)))
        obstaclespawns.append(np.random.randint(1, 19, size=(2)))
        for i in obstaclespawns:
            b0 = i
            b1 = (i[0] + 1, i[1])
            b2 = (i[0], i[1] + 1)
            b3 = (i[0] + 1, i[1] + 1)
            self.board[b0[0], b0[1]] = self.obstacleval
            self.board[b1[0], b1[1]] = self.obstacleval
            self.board[b2[0], b2[1]] = self.obstacleval
            self.board[b3[0], b3[1]] = self.obstacleval



    def SpawnSnake2(self):
        snakespawns = []
        for index, cell in np.ndenumerate(self.board):
            if cell == 0:
                snakespawns.append(index)
        self.snakespawn = random.choice(snakespawns)
        startx = self.snakespawn[0]
        starty = self.snakespawn[1]
        #self.board[startx, starty] = self.headval
        self.snake2 = Snake(startx, starty)
        return self.snake2


    def SpawnFood(self):
        foodspawns = []
        for index, cell in np.ndenumerate(self.board):
            if cell == 0:
                foodspawns.append(index)
        self.foodspawn = random.choice(foodspawns)
        self.board[self.foodspawn] = self.foodval

    def HeadPos(self, direction, headx, heady):
        if(0<headx<20) and (0<heady<20):
            if direction == 0:
                heady -= 1
            elif direction == 1:
                headx += 1
            elif direction == 2:
                heady += 1
            elif direction == 3:
                headx -= 1
        return (headx, heady)
    
    def CollisionCheck(self, direction, headx, heady):
        
        headx, heady = self.HeadPos(direction, headx, heady)

        if (headx == 0) or (headx == 20) or (heady == 0) or (heady == 20):
            return True
        elif self.board[heady, headx] == self.bodyval:
            return True
        elif self.board[heady, headx] == self.obstacleval:
            return True
        elif self.board[heady, headx] == self.headval:
            return True
        return False
   
    def Extractfeatures(self, headx, heady):
        
        for i in range(4):
            self.features[i] = self.CollisionCheck(i, headx, heady)
        #dist = self.FoodDirection(heady, headx)
        #self.features[4:] = 1 / (dist + 1)
        self.features[4:] = deepcopy(self.FoodDirection(heady, headx))
        return self.features
    
    def FoodDirection(self, heady, headx):
        foodvector = np.array(self.foodspawn) - np.array((heady, headx))
        foodfeatures = np.abs(foodvector)
        mandist = foodfeatures[0] + foodfeatures[1]
        return mandist

    def Move(self, direction, headx, heady, snakeindex):
        gameisover = False
        if not self.CollisionCheck(direction, headx, heady):
            reward = 0
            self.board[heady, headx] = self.bodyval

            potx, poty = self.HeadPos(direction, headx, heady)

            if self.board[poty, potx] == self.foodval:
                self.snakes[snakeindex].NewHead(potx, poty)
                self.board[poty, potx] = self.headval
                self.SpawnFood()
                self.length += 1 
                reward = 10000
            else:
                (oldtailx, oldtaily, newheadx, newheady) = self.snakes[snakeindex].Move(direction)
                self.board[oldtaily, oldtailx] = 0
                self.board[newheady, newheadx] = self.headval
        else:
            reward = -1000
            gameisover = True
        
        self.features = self.Extractfeatures(headx, heady)
        self.statevalue = np.dot(self.theta.T, self.features)
        return self.statevalue, self.features, reward, gameisover, self.length


    def BestMove(self, headx, heady):
        Wfeatures = deepcopy(self.Extractfeatures(headx, heady-1))
        Afeatures = deepcopy(self.Extractfeatures(headx-1, heady))
        Sfeatures = deepcopy(self.Extractfeatures(headx, heady+1))
        Dfeatures = deepcopy(self.Extractfeatures(headx+1, heady))
        Wscore = np.dot(self.theta.T, Wfeatures)
        Ascore = np.dot(self.theta.T, Afeatures)
        Sscore = np.dot(self.theta.T, Sfeatures)
        Dscore = np.dot(self.theta.T, Dfeatures)
        scores = []
        scores.append(Wscore)
        scores.append(Dscore)
        scores.append(Sscore)
        scores.append(Ascore)
        bestmove = np.argmax(scores)
        return bestmove


    def display(self):
        for i in range(22):
            print('-', end='')

        for i in range(20):
            print('\n|', end='')
            for j in range(20):
                if self.board[i, j] == 0:
                    print(' ', end='')
                elif self.board[i, j] == self.headval:
                    print('S', end='')
                elif self.board[i, j] == self.bodyval:
                    print('#', end='')
                elif self.board[i, j] == self.foodval:
                    print('F', end='')
                elif self.board[i, j] == self.obstacleval:
                    print('X', end='')
            print('|', end='')
        print()
        for i in range(22):
            print('-', end='')
        print()

In [337]:

weights = np.array([-5.6, -5.3, -3.7, -4.7, -116.8])
snakecount = int(input('number of snakes to be spawned'))
state = 1
snakeindex = 1
snakeisdead = np.zeros(snakecount)

while(state == 1):
    gamelog = []
    rewardlog = []
    valuelog = []

    game = Game(weights, snakecount)
    while True:
        

        for i in range(snakecount):


            (headx, heady) = game.snakes[i].Head().Pos()
            direction = game.BestMove(headx, heady)
            #direction = input("Input Direction (w,a,s,d or q to quit): ")
            if direction == 0:
                statevalue, features, reward, gameOver, score = game.Move(0, headx, heady, i)
            elif direction == 3:
                statevalue, features, reward, gameOver, score = game.Move(3, headx, heady, i)
            elif direction == 2:
                statevalue, features, reward, gameOver, score = game.Move(2, headx, heady, i)
            elif direction == 1:
                statevalue, features, reward, gameOver, score = game.Move(1, headx, heady, i)
            snakeisdead[i] = gameOver


            #rewardlog.append(deepcopy(reward))
            #gamelog.append(deepcopy(features))
            #valuelog.append(deepcopy(statevalue))
            print(f"features:{features}")
        game.display()

        if all(snakeisdead):
            print(f"Game Over")
            break
            
    state = int(input("press 1 to restart the game"))

number of snakes to be spawned4
features:[1 0 0 0 9]
features:[ 0  0  1  1 18]
features:[0 0 0 1 8]
features:[ 0  1  0  1 15]
----------------------
|                    |
|                    |
|              S XX  |
|            XX  XXS |
|            XX      |
|                    |
|                    |
|                    |
|                    |
|                    |
|                    |
|      XX F      S   |
|      XX            |
|                    |
|                    |
|                    |
|                XX  |
|       S        XX  |
|                    |
|                    |
----------------------
features:[1 0 0 0 8]
features:[ 0  0  1  1 17]
features:[0 0 0 1 7]
features:[ 0  0  0  1 14]
----------------------
|                    |
|                    |
|             S  XX  |
|            XX  XX  |
|            XX    S |
|                    |
|                    |
|                    |
|                    |
|                    |
|                    

In [338]:
x = [True, False, False, True]
y = np.zeros(4)
y[1] = True
all(y)

False